# Final Project - Task 1

## Requirements

1. <a href="#1">(20) Prepare 4 diﬀerent experimental settings using the CIFAR-100 dataset: Take 20%, 40%, 60%, and 100% of CIFAR-100 train images, respectively, and use them for train/val sets (train:val = 8:2, the same number of images per each class). For a common test set, use all the CIFAR-100 test images. Implement train/val/test dataloaders for the 4 diﬀerent experimental settings.</a>

2. <a href="#2">(10) For each setting, train VGG-16 and ResNet-50 from scratch and evaluate them. Plot the train/val curves and report test accuracy.</a>

3. <a href="#3">(20) Train VGG-16 and ResNet-50 from scratch using all training images of CIFAR10 (not CIFAR-100). Then, for each setting above, ﬁnetune the models and evaluate them. Plot the train/val curves and report test accuracy.</a>

4. <a href="#4">(20) Load VGG-16 and ResNet-50 pre-trained on the ImageNet dataset (using ’pretrained=True’ option). Then, for each setting above, ﬁne-tune the models and evaluate them. Plot the train/val curves and report test accuracy.</a>

5. <a href="#5">(5) Fill in the Table 1 below based on the results obtained above.</a>

6. <a href="#6">(25) Discuss the following points.</a>

  <a href="#6">(a) How is the performance of the funetuned models compared to that of the model trained from scratch? What makes the diﬀerence? </a>

  <a href="#6">(b) How is the performance of the ﬁnetuned models from CIFAR-10 compared to that of the ﬁnetuned models from ImageNet? What makes the diﬀerence? </a>

  <a href="#6">(c) Discuss the eﬀect of transfer learning in terms of the amount of training data. Explain why transfer learning is either more or less helpful.</a>



## Requirement 1

> <a name = "1">(20) Prepare 4 diﬀerent experimental settings using the **CIFAR-100 dataset**: Take 20%, 40%, 60%, and 100% of CIFAR-100 train images, respectively, and use them for train/val sets (**train:val = 8:2**, the same number of images per each class). For a common test set, use all the CIFAR-100 test images. Implement train/val/test dataloaders for the 4 diﬀerent experimental settings.</a>

The first step of course is download CIFAR100 dataset, similar with previous assignments. 

Then we should consider how to split such a dataset. Firstly, the test set would not need splitting, so I just generate the test data loader directly. As for splitting training set, we get to know the total length of train set, then we find the split point by referencing the rate we would like to do. 

Different from previous assignments, so far, we just have a new train set, which is part of the origin full train set. Then we should split the valid set. This part is the same with previous assignments. 

At the last, we push those split datasets to data loader to generate train data loader, valid data loader. 


In [5]:
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

def split(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2) 

  trainset_length = len(trainset)
  split_position = int(percentage * trainset_length)
  set_index = list(range(trainset_length))

  dataset = set_index[:split_position]

  trainset_length = len(dataset)
  split_position = int(1/4 * trainset_length)
  set_index = list(range(trainset_length))

  trainset = SubsetRandomSampler(set_index[split_position:])
  validset = SubsetRandomSampler(set_index[:split_position])

  trainloader = torch.utils.data.DataLoader(trainset, batch_size= 128, num_workers=2, shuffle=True)
  validloader = torch.utils.data.DataLoader(validset, batch_size= 128, num_workers=2, shuffle=False)

  return trainset, validset, trainloader, validloader

trainset20, validset20, trainloader20, validloader20 = split(0.2)

trainset40, validset40, trainloader40, validloader40 = split(0.4)

trainset60, validset60, trainloader60, validloader60 = split(0.6)

trainset, validset, trainloader, validloader = split(1)

0it [00:00, ?it/s]

169009152it [00:02, 69323926.54it/s]                               


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Requierement 2

> <a name = "2">(10) For each setting, train **VGG-16** and **ResNet-50** from scratch and evaluate them. Plot the train/val curves and report test accuracy.</a>

Just as requirement, we choose VGG16 as our model. Besides, we choose cross entropy loss as our loss function and SGD as our optimizer. 

The creation of training is not so different from previous works, including introducing data, calculating grad, backward spreading, and then calculating the loss of this round. In the meanwhile, record every step’s loss for graphing. 


In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def split(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)
  
  return trainloader, validloader, testloader

trainloader20, validloader20, testloader20 = split(0.2)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = models.vgg16(pretrained=False)
if torch.cuda.is_available():
  model = model.cuda()  

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

epoches = 10
step = 0
running_loss = 0
train_losses, test_losses = [], []

for epoch in range(epoches):
  for inputs, labels in trainloader20:
    inputs = inputs.cuda()
    labels = labels.cuda()
    step += 1

    optimizer.zero_grad()

    logps = model.forward(inputs)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * inputs.size(0)

    if step % 100 == 0:
      test_loss = 0
      accuracy = 0
      model.eval()
      with torch.no_grad():
        for inputs, labels in validloader20:
          inputs = inputs.cuda()
          labels = labels.cuda()
          logps = model.forward(inputs)
          batch_loss = criterion(logps, labels)
          test_loss += batch_loss.item()*inputs.size(0)

          ps = torch.exp(logps)
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == labels.view(*top_class.shape)
          accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
       
        print(type(trainloader20))
        train_losses.append(running_loss/len(trainloader20))
        test_losses.append(test_loss/len(validloader20))                    
        print(f"Epoch {epoch+1}/{epoches}.. "
              f"Train loss: {running_loss/100:.3f}.. "
              f"Val loss: {test_loss/len(validloader20):.3f}.. "
              f"Val accuracy: {accuracy/len(validloader20):.3f}")
        running_loss = 0
        model.train()

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

accuracy = 0
total_number = 0

with torch.no_grad():
    for data in testloader20:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_number += labels.size(0)
        accuracy += (predicted == labels).sum().item()

print('Test Accuracy: ', (100 * accuracy / total_number))


Modify all the processes to a function. 

In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def task1_question2_vgg16(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model = models.vgg16(pretrained=False)
  if torch.cuda.is_available():
    model = model.cuda()  

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

  epoches = 10
  step = 0
  running_loss = 0
  train_losses, test_losses = [], []

  for epoch in range(epoches):
    for inputs, labels in trainloader:
      inputs = inputs.cuda()
      labels = labels.cuda()
      step += 1

      optimizer.zero_grad()

      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      if step % 100 == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
          for inputs, labels in validloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()*inputs.size(0)

            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
          print(type(trainloader))
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(validloader))                    
          print(f"Epoch {epoch+1}/{epoches}.. "
                f"Train loss: {running_loss/100:.3f}.. "
                f"Val loss: {test_loss/len(validloader):.3f}.. "
                f"Val accuracy: {accuracy/len(validloader):.3f}")
          running_loss = 0
          model.train()

  plt.plot(train_losses, label='Training loss')
  plt.plot(test_losses, label='Validation loss')
  plt.legend(frameon=False)
  plt.show()

  accuracy = 0
  total_number = 0

  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.cuda()
          labels = labels.cuda()
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_number += labels.size(0)
          accuracy += (predicted == labels).sum().item()

  print('Test Accuracy: ', (100 * accuracy / total_number))


In [0]:
task1_question2_vgg16(0.4)
task1_question2_vgg16(0.6)
task1_question2_vgg16(1.0)

Then here is the ResNet model. 

In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def task1_question2_resnet50(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model = models.resnet50(pretrained=False)
  if torch.cuda.is_available():
    model = model.cuda()  

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

  epoches = 10
  step = 0
  running_loss = 0
  train_losses, test_losses = [], []

  for epoch in range(epoches):
    for inputs, labels in trainloader:
      inputs = inputs.cuda()
      labels = labels.cuda()
      step += 1

      optimizer.zero_grad()

      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      if step % 100 == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
          for inputs, labels in validloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()*inputs.size(0)

            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
          print(type(trainloader))
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(validloader))                    
          print(f"Epoch {epoch+1}/{epoches}.. "
                f"Train loss: {running_loss/100:.3f}.. "
                f"Val loss: {test_loss/len(validloader):.3f}.. "
                f"Val accuracy: {accuracy/len(validloader):.3f}")
          running_loss = 0
          model.train()

  plt.plot(train_losses, label='Training loss')
  plt.plot(test_losses, label='Validation loss')
  plt.legend(frameon=False)
  plt.show()

  accuracy = 0
  total_number = 0

  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.cuda()
          labels = labels.cuda()
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_number += labels.size(0)
          accuracy += (predicted == labels).sum().item()

  print('Test Accuracy: ', (100 * accuracy / total_number))

In [0]:
task1_question2_resnet50(0.2)
task1_question2_resnet50(0.4)
task1_question2_resnet50(0.6)
task1_question2_resnet50(1.0)

## Requirement 3

> <a name = "3">(20) Train VGG-16 and ResNet-50 from scratch using all training images of CIFAR10 (not CIFAR-100). Then, for each setting above, ﬁnetune the models and evaluate them. Plot the train/val curves and report test accuracy.</a>

As the requirement, we should change the dataset to CIFAR-10 and then train again. 

The models we used are the same, so we would not need to change the main training and creating model method. 


In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def task1_question3_vgg16(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model = models.vgg16(pretrained=False)
  if torch.cuda.is_available():
    model = model.cuda()  

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

  epoches = 2
  step = 0
  running_loss = 0
  train_losses, test_losses = [], []

  for epoch in range(epoches):
    for inputs, labels in trainloader:
      inputs = inputs.cuda()
      labels = labels.cuda()
      step += 1

      optimizer.zero_grad()

      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      if step % 100 == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
          for inputs, labels in validloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()*inputs.size(0)

            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
          print(type(trainloader))
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(validloader))                    
          print(f"Epoch {epoch+1}/{epoches}.. "
                f"Train loss: {running_loss/100:.3f}.. "
                f"Val loss: {test_loss/len(validloader):.3f}.. "
                f"Val accuracy: {accuracy/len(validloader):.3f}")
          running_loss = 0
          model.train()

  plt.plot(train_losses, label='Training loss')
  plt.plot(test_losses, label='Validation loss')
  plt.legend(frameon=False)
  plt.show()

  accuracy = 0
  total_number = 0

  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.cuda()
          labels = labels.cuda()
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_number += labels.size(0)
          accuracy += (predicted == labels).sum().item()

  print('Test Accuracy: ', (100 * accuracy / total_number))


In [0]:
task1_question3_vgg16(0.2)
task1_question3_vgg16(0.4)
task1_question3_vgg16(0.6)
task1_question3_vgg16(1.0)

Then here is the ResNet-50 model. 

In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def task1_question3_resnet50(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model = models.resnet50(pretrained=False)
  if torch.cuda.is_available():
    model = model.cuda()  

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

  epoches = 10
  step = 0
  running_loss = 0
  train_losses, test_losses = [], []

  for epoch in range(epoches):
    for inputs, labels in trainloader:
      inputs = inputs.cuda()
      labels = labels.cuda()
      step += 1

      optimizer.zero_grad()

      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      if step % 100 == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
          for inputs, labels in validloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()*inputs.size(0)

            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
          print(type(trainloader))
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(validloader))                    
          print(f"Epoch {epoch+1}/{epoches}.. "
                f"Train loss: {running_loss/100:.3f}.. "
                f"Val loss: {test_loss/len(validloader):.3f}.. "
                f"Val accuracy: {accuracy/len(validloader):.3f}")
          running_loss = 0
          model.train()

  plt.plot(train_losses, label='Training loss')
  plt.plot(test_losses, label='Validation loss')
  plt.legend(frameon=False)
  plt.show()

  accuracy = 0
  total_number = 0

  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.cuda()
          labels = labels.cuda()
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_number += labels.size(0)
          accuracy += (predicted == labels).sum().item()

  print('Test Accuracy: ', (100 * accuracy / total_number))


In [0]:
task1_question3_resnet50(0.2)
task1_question3_resnet50(0.4)
task1_question3_resnet50(0.6)
task1_question3_resnet50(1.0)

## Requirement 4
 >  <a name = "4">(20) Load VGG-16 and ResNet-50 pre-trained on the ImageNet dataset (using ’pretrained=True’ option). Then, for each setting above, ﬁne-tune the models and evaluate them. Plot the train/val curves and report test accuracy.</a>

Now we add the ImageNet dataset. Train those two models again to find the result. 

In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def task1_question4_vgg16(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model = models.vgg16(pretrained=True)
  if torch.cuda.is_available():
    model = model.cuda()  

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

  epoches = 10
  step = 0
  running_loss = 0
  train_losses, test_losses = [], []

  for epoch in range(epoches):
    for inputs, labels in trainloader:
      inputs = inputs.cuda()
      labels = labels.cuda()
      step += 1

      optimizer.zero_grad()

      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      if step % 100 == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
          for inputs, labels in validloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()*inputs.size(0)

            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
          print(type(trainloader))
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(validloader))                    
          print(f"Epoch {epoch+1}/{epoches}.. "
                f"Train loss: {running_loss/100:.3f}.. "
                f"Val loss: {test_loss/len(validloader):.3f}.. "
                f"Val accuracy: {accuracy/len(validloader):.3f}")
          running_loss = 0
          model.train()

  plt.plot(train_losses, label='Training loss')
  plt.plot(test_losses, label='Validation loss')
  plt.legend(frameon=False)
  plt.show()

  accuracy = 0
  total_number = 0

  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.cuda()
          labels = labels.cuda()
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_number += labels.size(0)
          accuracy += (predicted == labels).sum().item()

  print('Test Accuracy: ', (100 * accuracy / total_number))

In [0]:
task1_question4_vgg16(0.2)
task1_question4_vgg16(0.4)
task1_question4_vgg16(0.6)
task1_question4_vgg16(1.0)

In [0]:
import sys
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models

def task1_question4_resnet50(percentage):
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
  testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

  testloader = torch.utils.data.DataLoader(testset, batch_size= 128, num_workers=2)

  total_length=int(percentage* len(trainset))
  total_dataset,no_train_dataset=torch.utils.data.random_split(trainset,(total_length,len(trainset)-total_length))
  train_length=int(0.8* len(total_dataset))
  train_dataset,val_dataset=torch.utils.data.random_split(total_dataset,(train_length,len(total_dataset)-train_length))

  trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
  validloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=2)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model = models.resnet50(pretrained=True)
  if torch.cuda.is_available():
    model = model.cuda()  

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)

  epoches = 2
  step = 0
  running_loss = 0
  train_losses, test_losses = [], []

  for epoch in range(epoches):
    for inputs, labels in trainloader:
      inputs = inputs.cuda()
      labels = labels.cuda()
      step += 1

      optimizer.zero_grad()

      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      if step % 100 == 0:
        test_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
          for inputs, labels in validloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()*inputs.size(0)

            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
          print(type(trainloader))
          train_losses.append(running_loss/len(trainloader))
          test_losses.append(test_loss/len(validloader))                    
          print(f"Epoch {epoch+1}/{epoches}.. "
                f"Train loss: {running_loss/100:.3f}.. "
                f"Val loss: {test_loss/len(validloader):.3f}.. "
                f"Val accuracy: {accuracy/len(validloader):.3f}")
          running_loss = 0
          model.train()

  plt.plot(train_losses, label='Training loss')
  plt.plot(test_losses, label='Validation loss')
  plt.legend(frameon=False)
  plt.show()

  accuracy = 0
  total_number = 0

  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.cuda()
          labels = labels.cuda()
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total_number += labels.size(0)
          accuracy += (predicted == labels).sum().item()

  print('Test Accuracy: ', (100 * accuracy / total_number))

In [0]:
task1_question4_resnet50(0.2)
task1_question4_resnet50(0.4)
task1_question4_resnet50(0.6)
task1_question4_resnet50(1.0)

## Requirement 5

>  <a name = "5">(5) Fill in the Table 1 below based on the results obtained above.</a>

N.B. The radioes here are rounding numbers.

|the percentage of using train data|20%|40%|60%|100%|
|-|-|-|-|-|
|VGG-16 trained from scratch|1%|3%|4%|5%|
|VGG-16 fine-tuned from CIFAR-19|8%|15%|23%|29%|
|VGG-16 fine-turned from ImageNet|36%|41%|44%|49%|
|ResNet-50 trained from scratch|10%|15%|16%|21%|
|ResNet-50 finetuned from FICAR-10|11%|17%|20%|24%|
|ResNet-50 finetuned from ImageNet|31%|40%|45%|52%|

## Requirement 6

>  <a name = "6">(25) Discuss the following points.</a>
>
>  <a name = "6">(a) How is the performance of the funetuned models compared to that of the model trained from scratch? What makes the diﬀerence? </a>
>    
>  <a name = "6">(b) How is the performance of the ﬁnetuned models from CIFAR-10 compared to that of the ﬁnetuned models from ImageNet? What makes the diﬀerence? </a>
>  
>  <a name = "6">(c) Discuss the eﬀect of transfer learning in terms of the amount of training data. Explain why transfer learning is either more or less helpful.</a>

Answer: 

Question (a): the performance of funetuned model is better than the one of scratch. The reason would be the big dataset, much more data can help model have better generalization abilities since at this time, the overfitting problem would be mitigated somewhat. 

Question (b): the performance of ImageNet model is better than the one of the finetuned model. I believe the reason is similar to the former one: the ImageNet have much larger data than CIFAR and the model VGG is trained based on CIFAR dataset, as a result, the larger dataset brings better generalization abilities with reducing overfitting. 

Question (c): Of course transfer learning can increase the size of dataset. Just as I discussed in the former two questions, larger dataset can bring better test performance moreor less. 